## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-30-02-22-00 +0000,wsj,Trump Says U.S. Will Begin Testing Nuclear Wea...,https://www.wsj.com/world/china/trump-says-u-s...
1,2025-10-30-02-19-13 +0000,startribune,Twins choose Derek Shelton as their next manager,https://www.startribune.com/twins-derek-shelto...
2,2025-10-30-02-15-01 +0000,wapo,Trump directs Pentagon to test nuclear weapons...,https://www.washingtonpost.com/politics/2025/1...
3,2025-10-30-02-11-09 +0000,startribune,Live updates from girls soccer state semifinal...,https://www.startribune.com/scores-and-schedul...
4,2025-10-30-02-09-46 +0000,nyt,California Animal Rights Activist Convicted in...,https://www.nytimes.com/2025/10/29/us/animal-r...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
0,trump,50
108,new,27
1,will,17
181,hurricane,16
183,melissa,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
2,2025-10-30-02-15-01 +0000,wapo,Trump directs Pentagon to test nuclear weapons...,https://www.washingtonpost.com/politics/2025/1...,113
294,2025-10-29-14-38-11 +0000,nypost,Trump’s hot mic moment teases plans for summit...,https://nypost.com/2025/10/29/us-news/trump-he...,105
178,2025-10-29-20-09-08 +0000,wapo,"Trump rebukes Putin for testing new missiles, ...",https://www.washingtonpost.com/world/2025/10/2...,104
396,2025-10-29-06-52-00 +0000,wsj,Trump Expects to Sign China Trade Deal at Xi S...,https://www.wsj.com/economy/trade/trump-expect...,104
115,2025-10-29-21-43-10 +0000,nypost,Trump confident cease-fire will hold after Isr...,https://nypost.com/2025/10/29/world-news/trump...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
2,113,2025-10-30-02-15-01 +0000,wapo,Trump directs Pentagon to test nuclear weapons...,https://www.washingtonpost.com/politics/2025/1...
77,73,2025-10-29-22-57-52 +0000,nypost,Jack Ciattarelli says NJ will welcome any New ...,https://nypost.com/2025/10/29/us-news/jack-cia...
18,69,2025-10-30-01-32-00 +0000,wsj,Federal Reserve Chair Jerome Powell delivered ...,https://www.wsj.com/economy/central-banking/fe...
152,61,2025-10-29-20-56-14 +0000,wapo,Hurricane Melissa hits Cuba; has killed at lea...,https://www.washingtonpost.com/weather/2025/10...
222,49,2025-10-29-18-09-32 +0000,nypost,Fed cuts interest rates by quarter point for s...,https://nypost.com/2025/10/29/business/fed-cut...
227,49,2025-10-29-18-05-17 +0000,wapo,Report: Government shutdown will cost the econ...,https://www.washingtonpost.com/business/2025/1...
282,45,2025-10-29-15-08-02 +0000,nypost,Nvidia becomes first $5T company powered by AI...,https://nypost.com/2025/10/29/business/nvidia-...
267,45,2025-10-29-16-04-58 +0000,bbc,"Israeli strikes in Gaza kill 104, health minis...",https://www.bbc.com/news/articles/cgjdy5eevn2o...
340,45,2025-10-29-11-30-50 +0000,nypost,"US, South Korea reach deal for Seoul to buy Am...",https://nypost.com/2025/10/29/world-news/presi...
87,41,2025-10-29-22-41-41 +0000,nypost,NYC maniac attacked model ex’s 12-year daughte...,https://nypost.com/2025/10/29/us-news/nyc-mani...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
